In [1]:
import os
import time
import hashlib
import requests
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from PIL import Image
from io import BytesIO
import csv


In [2]:
chrome_options = Options()
chrome_options.add_argument('--headless')  # Run in headless mode
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)


In [5]:
def scroll_to_bottom(driver):
    """Scrolls to the bottom of the page to load all images."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for new images to load
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def extract_image_urls(driver, query, max_images):
    """Extracts image URLs from Pixabay based on the search query."""
    search_url = f"https://pixabay.com/images/search/{query}/"
    driver.get(search_url)
    scroll_to_bottom(driver)
    img_elements = driver.find_elements(By.CSS_SELECTOR, "img")
    img_urls = set()
    for img in img_elements:
        src = img.get_attribute('src')
        if src:
            img_urls.add(src)
        if len(img_urls) >= max_images:
            break
    return img_urls

def download_image(url, output_dir, category):
    """Downloads an image and saves it to the specified directory."""
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))
            file_hash = hashlib.md5(response.content).hexdigest()
            file_extension = image.format.lower()
            file_name = f"{category}_{file_hash}.{file_extension}"
            file_path = os.path.join(output_dir, file_name)
            image.save(file_path)
            return file_path
    except Exception as e:
        print(f"Failed to download {url}: {e}")
    return None


In [9]:
query = 'bicycle'  # Replace with your search term
output_dir = 'scraped_images'
max_images = 500

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

img_urls = extract_image_urls(driver, query, max_images)
print(f"Found {len(img_urls)} images for query '{query}'.")

with open('image_metadata.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['category', 'image_url', 'download_time', 'file_path'])

    for url in img_urls:
        file_path = download_image(url, output_dir, query)
        if file_path:
            writer.writerow([query, url, datetime.now(), file_path])

print(img_urls)
driver.quit()

MaxRetryError: HTTPConnectionPool(host='localhost', port=12519): Max retries exceeded with url: /session/2fab6ff0b4fa3cf04176e2e14c136dd7/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001AD904FE5A0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [8]:
driver.quit()